# Create Granger DataFrame

## Imports

In [55]:
# Pandas is an open source data analysis and manipulation tool
import pandas as pd
#adf test
from statsmodels.tsa.stattools import adfuller
#granger causality test
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import kpss
import numpy as np

## Load DataFrame

In [5]:
#read dataframe
Occurence_df = pd.read_csv("/Users/jan/Documents/Python_Projects/Bachelorthesis/Analysis/DataFrames/Combined_Dataframe_11042022.csv")
#set correact_datatypes for dataframe
Occurence_df = set_correct_datatypes(Occurence_df)
Occurence_df = Occurence_df.groupby("KeyWord").apply(smoothen_timeseries)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 110380 entries, 0 to 110379
Data columns (total 14 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   KeyWord                                       110380 non-null  object        
 1   date                                          110380 non-null  datetime64[ns]
 2   Occurence_in_News                             110380 non-null  int64         
 3   normalized_Occurence_in_News                  110380 non-null  float64       
 4   Occurence_in_Google                           110380 non-null  int64         
 5   normalized_Occurence_in_Google                110380 non-null  float64       
 6   Occurence_in_Wikipedia                        110380 non-null  int64         
 7   normalized_Occurence_in_Wikipedia             110380 non-null  float64       
 8   smoothened_normalized_Occurence_in_News       105949 n

## Helper

In [13]:
#save dataframe as csv
def saveDF_as_CSV(dataframe, filename):
    dataframe.to_csv("/Users/jan/Documents/Python_Projects/Bachelorthesis/Analysis/DataFrames/"+ filename +".csv",index=False)

def smoothen_timeseries(dataframe):
    df = dataframe.copy()
    window_size = 7
    normalized_Occurence_column_names = ["normalized_Occurence_in_News","normalized_Occurence_in_Google",
                                         "normalized_Occurence_in_Wikipedia"]
    Occurence_column_names = ["Occurence_in_News","Occurence_in_Google","Occurence_in_Wikipedia"]

    Column_names = normalized_Occurence_column_names + Occurence_column_names

    for column_name in Column_names:
        # smoothing Occurence

        data = df[column_name].rolling(window_size).sum()
        dataframe["smoothened_"+column_name] = data
    return dataframe

def set_correct_datatypes(dataframe):
    dataframe.KeyWord.astype(str)
    dataframe["date"] = pd.to_datetime(dataframe["date"], format='%Y-%m-%d')
    return dataframe



## Create DataFrame

This function takes time series data of the Occurence Dataframe, performs ADF Test, performs the grangercausalitytest and saves all results in a dataframe

!!! Data is not corrected for stationarity !!!

In [51]:
# TODO pass ammount of lags to function and create column names automatically
def add_lowest_lag_column(df):
    wiki_list = df[["ssr_ftest_pValue_lag1" ,'ssr_ftest_pValue_lag2' , "ssr_ftest_pValue_lag3" , "ssr_ftest_pValue_lag4" , "ssr_ftest_pValue_lag5" , "ssr_ftest_pValue_lag6" , "ssr_ftest_pValue_lag7"]] .values.tolist()
    result_lst = []
    for row in wiki_list:
        min_value = min(row)
        if min_value <= 0.05:
            min_index = row.index(min_value) #not zero based
            result_lst.append(min_index+1)
        else:
            result_lst.append(np.nan)
    df["lowest_lag"] = result_lst
    return df

In [56]:
granger_test_df = Occurence_df[["KeyWord","date", "Occurence_in_News", "Occurence_in_Wikipedia",]]
granger_test_df= granger_test_df.set_index("date")
# add missing dates
#get first and last date
date_range = pd.date_range(granger_test_df.index.min(), granger_test_df.index.max(), freq='D')

return_list = []

News_grangerCauses_Wikipedia = ['Occurence_in_Wikipedia', 'Occurence_in_News']
Wikipedia_grangerCauses_News = ['Occurence_in_News', 'Occurence_in_Wikipedia']

grouped_by_keyword = granger_test_df.groupby("KeyWord")

not_testable_keywords = []

for key, item in grouped_by_keyword:
    try:
        return_dict = {}
        grouped_keyWord = grouped_by_keyword.get_group(key)

        #get name of keyword
        keyWord = grouped_keyWord["KeyWord"].iloc[0]
        return_dict["keyword_name"] = keyWord
        print(keyWord)

        #add missing dates
        grouped_keyWord.index = pd.DatetimeIndex(grouped_keyWord.index)
        grouped_keyWord = grouped_keyWord.reindex(date_range, fill_value=0)


        #ADF test
        #print(f'ADF Test: {keyword_name} time series')
        adf_test_result_wikipedia = adfuller(pd.DataFrame(grouped_keyWord['Occurence_in_Wikipedia']).values)
        adf_test_result_news      = adfuller(pd.DataFrame(grouped_keyWord['Occurence_in_News']).values)

        #save ADF test results
        #wikipedia
        return_dict["ADF_Statistics_wiki"] = adf_test_result_wikipedia[0]
        return_dict["ADF_p_value_wiki"] = adf_test_result_wikipedia[1]
        return_dict["ADF_critical_values_wiki"] = adf_test_result_wikipedia[4]
        #news
        return_dict["ADF_Statistics_news"] = adf_test_result_news[0]
        return_dict["ADF_p_value_news"] = adf_test_result_news[1]
        return_dict["ADF_critical_values_news"] = adf_test_result_news[4]


        # KPSS Test
        kpss_statistic_wiki, kpss_pValue_wiki, kpss_nLags_wiki, kpss_criticalValues_wiki = kpss(pd.DataFrame(grouped_keyWord['Occurence_in_Wikipedia']).values)
        kpss_statistic_news, kpss_pValue_news, kpss_nLags_news, kpss_criticalValues_news = kpss(pd.DataFrame(grouped_keyWord['Occurence_in_News']).values)

        number_of_lags = 7

        granger_test_result = grangercausalitytests(grouped_keyWord[Wikipedia_grangerCauses_News], maxlag=number_of_lags)

        test_name='ssr_ftest'
        #column of result
        column = 1

        p_values = [round(granger_test_result[i+1][0][test_name][column],4) for i in range(number_of_lags)]
        #print(p_values)
        for lag in range(number_of_lags):
            # lags are not zero based
            lag_not_zero_based = lag+1
            if column == 1:
                column = "pValue"
            key = f"{test_name}_{column}_lag{lag_not_zero_based}"
            value = p_values[lag]
            return_dict[key] = value
        return_list.append(return_dict)

    except:
        print(f"Error! {keyWord}" )
        not_testable_keywords.append(keyWord)
        pass
df = pd.DataFrame(return_list)
df = add_lowest_lag_column(df)

saveDF_as_CSV(df, "Wikipedia_grangerCauses_News___NOT_Stationary_DataFrame")

2G

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2.0089  , p=0.1585  , df_denom=150, df_num=1
ssr based chi2 test:   chi2=2.0491  , p=0.1523  , df=1
likelihood ratio test: chi2=2.0355  , p=0.1537  , df=1
parameter F test:         F=2.0089  , p=0.1585  , df_denom=150, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.0287  , p=0.3600  , df_denom=147, df_num=2
ssr based chi2 test:   chi2=2.1274  , p=0.3452  , df=2
likelihood ratio test: chi2=2.1127  , p=0.3477  , df=2
parameter F test:         F=1.0287  , p=0.3600  , df_denom=147, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=4.4891  , p=0.0048  , df_denom=144, df_num=3
ssr based chi2 test:   chi2=14.1220 , p=0.0027  , df=3
likelihood ratio test: chi2=13.5001 , p=0.0037  , df=3
parameter F test:         F=4.4891  , p=0.0048  , df_denom=144, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=4.5695  , p=0.001

/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value retu

ssr based F test:         F=0.7515  , p=0.6093  , df_denom=135, df_num=6
ssr based chi2 test:   chi2=4.9429  , p=0.5512  , df=6
likelihood ratio test: chi2=4.8621  , p=0.5616  , df=6
parameter F test:         F=0.7515  , p=0.6093  , df_denom=135, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.5191  , p=0.8189  , df_denom=132, df_num=7
ssr based chi2 test:   chi2=4.0467  , p=0.7744  , df=7
likelihood ratio test: chi2=3.9920  , p=0.7807  , df=7
parameter F test:         F=0.5191  , p=0.8189  , df_denom=132, df_num=7
3G

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.7854  , p=0.3769  , df_denom=150, df_num=1
ssr based chi2 test:   chi2=0.8011  , p=0.3708  , df=1
likelihood ratio test: chi2=0.7990  , p=0.3714  , df=1
parameter F test:         F=0.7854  , p=0.3769  , df_denom=150, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.7040  , p=0.4962  , df_denom=147, df_num=2
ssr based chi2 tes

/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length 


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.5220  , p=0.4711  , df_denom=150, df_num=1
ssr based chi2 test:   chi2=0.5325  , p=0.4656  , df=1
likelihood ratio test: chi2=0.5316  , p=0.4660  , df=1
parameter F test:         F=0.5220  , p=0.4711  , df_denom=150, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.5792  , p=0.5616  , df_denom=147, df_num=2
ssr based chi2 test:   chi2=1.1979  , p=0.5494  , df=2
likelihood ratio test: chi2=1.1932  , p=0.5507  , df=2
parameter F test:         F=0.5792  , p=0.5616  , df_denom=147, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.3858  , p=0.7634  , df_denom=144, df_num=3
ssr based chi2 test:   chi2=1.2135  , p=0.7498  , df=3
likelihood ratio test: chi2=1.2087  , p=0.7509  , df=3
parameter F test:         F=0.3858  , p=0.7634  , df_denom=144, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.3716  , p=0.8286  

...

ssr based F test:         F=0.3679  , p=0.8982  , df_denom=135, df_num=6
ssr based chi2 test:   chi2=2.4197  , p=0.8773  , df=6
likelihood ratio test: chi2=2.4002  , p=0.8795  , df=6
parameter F test:         F=0.3679  , p=0.8982  , df_denom=135, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.3773  , p=0.9143  , df_denom=132, df_num=7
ssr based chi2 test:   chi2=2.9410  , p=0.8904  , df=7
likelihood ratio test: chi2=2.9119  , p=0.8930  , df=7
parameter F test:         F=0.3773  , p=0.9143  , df_denom=132, df_num=7
Zuschauer

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=1.3938  , p=0.2396  , df_denom=150, df_num=1
ssr based chi2 test:   chi2=1.4217  , p=0.2331  , df=1
likelihood ratio test: chi2=1.4151  , p=0.2342  , df=1
parameter F test:         F=1.3938  , p=0.2396  , df_denom=150, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.6737  , p=0.5114  , df_denom=147, df_num=2
ssr based c

/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length 

ssr based F test:         F=0.0042  , p=0.9483  , df_denom=150, df_num=1
ssr based chi2 test:   chi2=0.0043  , p=0.9477  , df=1
likelihood ratio test: chi2=0.0043  , p=0.9477  , df=1
parameter F test:         F=0.0042  , p=0.9483  , df_denom=150, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.8327  , p=0.4369  , df_denom=147, df_num=2
ssr based chi2 test:   chi2=1.7220  , p=0.4227  , df=2
likelihood ratio test: chi2=1.7123  , p=0.4248  , df=2
parameter F test:         F=0.8327  , p=0.4369  , df_denom=147, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.8628  , p=0.4620  , df_denom=144, df_num=3
ssr based chi2 test:   chi2=2.7141  , p=0.4378  , df=3
likelihood ratio test: chi2=2.6900  , p=0.4419  , df=3
parameter F test:         F=0.8628  , p=0.4620  , df_denom=144, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.7882  , p=0.5347  , df_denom=141, df_num=4
ssr based chi2 test: 

/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/Users/jan/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1875: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length 

ssr based F test:         F=1.2697  , p=0.2754  , df_denom=135, df_num=6
ssr based chi2 test:   chi2=8.3519  , p=0.2134  , df=6
likelihood ratio test: chi2=8.1247  , p=0.2291  , df=6
parameter F test:         F=1.2697  , p=0.2754  , df_denom=135, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.9817  , p=0.4474  , df_denom=132, df_num=7
ssr based chi2 test:   chi2=7.6526  , p=0.3642  , df=7
likelihood ratio test: chi2=7.4600  , p=0.3826  , df=7
parameter F test:         F=0.9817  , p=0.4474  , df_denom=132, df_num=7
Überfall

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=3.4372  , p=0.0657  , df_denom=150, df_num=1
ssr based chi2 test:   chi2=3.5060  , p=0.0611  , df=1
likelihood ratio test: chi2=3.4664  , p=0.0626  , df=1
parameter F test:         F=3.4372  , p=0.0657  , df_denom=150, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.9372  , p=0.0216  , df_denom=147, df_num=2
ssr based ch